In [1]:
import pandas as pd
import torch
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from skimage import io, transform
import matplotlib.pyplot as plt
from torch.autograd import Variable
import numpy as np
import torch.nn as nn
from skimage.transform import rescale, resize, downscale_local_mean
from torchvision import datasets, transforms
from PIL import Image
from tqdm import tqdm
import h5py
import pickle
from transformers import *
from sklearn.externals import joblib
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from tensorboardX import SummaryWriter

/home/nikepupu/anaconda3/envs/mine/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nikepupu/anaconda3/envs/mine/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nikepupu/anaconda3/envs/mine/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/nikepupu/anaconda3/envs/mine/lib/python3

In [2]:
class CAD120Dataset(Dataset):
    def __init__(self, feature_file,  caption_file , transform=None):

        
        self.caption = joblib.load(caption_file)
        self.caption = [ cap.permute(1,0,2) for cap in self.caption ]
        self.features = joblib.load(feature_file)
        
    
    def __len__(self):
        return len(self.caption)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        features = self.features[idx]
        captions = self.caption[idx]
        
       
        sample = {"features": features, "captions":captions}

        return sample

In [3]:
device = 'cuda:0'
class Net(nn.Module):
    def __init__(self, input_size_image, input_size_word_embedding, 
                hidden_size, output_size,num_layers=1):
        super(Net, self).__init__()
        self.input_size_image = input_size_image
        self.input_size_word_embedding = input_size_word_embedding
        self.hidden_size = hidden_size
        
        self.num_layers = num_layers
        self.lstm_image = nn.LSTM(input_size_image, hidden_size, num_layers,  bidirectional=True)
        self.lstm_caption = nn.LSTM(input_size_word_embedding, hidden_size, num_layers,  bidirectional=True)
      
    
    def forward(self, image_features, caption_features):
        # Set initial states
        
        img_batch_len = len(image_features)
        caption_batch_len = len(caption_features)
        
 
       
        h0 = torch.zeros(self.num_layers*2, 1, self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers*2, 1, self.hidden_size).to(device)
        
        h1 = torch.zeros(self.num_layers*2, 1, self.hidden_size).to(device)
        c1 = torch.zeros(self.num_layers*2, 1, self.hidden_size).to(device)
        OUT0 = []
        OUT1 = []
        for i in range(img_batch_len):
            #print(image_features[i].shape)
            h0 = torch.zeros(self.num_layers*2, 1, self.hidden_size).to(device)
            c0 = torch.zeros(self.num_layers*2, 1, self.hidden_size).to(device)
            for j in range(image_features[i].shape[0]):
                out0, (h0,c0) = self.lstm_image(image_features[i][j].view(1,1,-1), (h0,c0))  # out: tensor of shape (batch_size, seq_length, hidden_size*2)
            OUT0.append(out0)
                
            
        for i in range(caption_batch_len):
            h1 = torch.zeros(self.num_layers*2, 1, self.hidden_size).to(device)
            c1 = torch.zeros(self.num_layers*2, 1, self.hidden_size).to(device)
            for j in range(caption_features[i].shape[0]):
                out1, (h1, c1) = self.lstm_caption(caption_features[i][j].view(1,1,-1), (h1, c1))
            OUT1.append(out1)
        

        OUT0 = torch.stack(OUT0)
        OUT1 = torch.stack(OUT1)
        OUT0 = OUT0.view(OUT0.shape[0],-1)
        OUT1 = OUT1.view(OUT1.shape[0],-1)
        return OUT0,OUT1



In [4]:
model = Net(512*4*4, 768, 256, 10,1).to(device)
writer = SummaryWriter()

In [5]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, b in enumerate(train_loader):
        imgs = b[0]
        captions = b[1]
        
        img_feature, caption_feature  = imgs, [caption.to(device) for caption in captions]
        
        optimizer.zero_grad()
        model(img_feature, caption_feature)
        h_image, h_caption = model(img_feature, caption_feature)

    
    
        l = h_image.shape[0]
        
        loss = 0
        S_image = -1*torch.ones((l,1)).cuda()
        S_caption = -1*torch.ones((l,1)).cuda()
        accuracy = 0
        
        for i in range(l):
            index_image = 0
            for j in range(l):
                if S_image[i] <  h_image[i].dot(h_caption[j]):
                    S_image[i] =  h_image[i].dot(h_caption[j])
                    index_image = j
                    
            if i == index_image:
                accuracy += 1.0
            loss += max(0,S_image[i]- h_image[i].dot(h_caption[i])+1)
        
        
        for i in range(l):
            index_caption = 0
            for j in range(l):    
                if S_caption[i] <  h_caption[i].dot(h_image[j]):
                    S_caption[i] =  h_caption[i].dot(h_image[j])
                    index_caption = j
                    
            if i == index_caption:
                accuracy += 1.0
            loss += max(0,S_caption[i]-h_image[i].dot(h_caption[i])+1)
        
        accuracy /= float(2*l)
        loss /= float(l)
            
            
        loss.backward()
        optimizer.step()
        #if batch_idx % 16 == 0:
        
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(imgs), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
        

        return loss.item(), accuracy

In [6]:
def test(model, device, test_loader,epoch):
    model.eval()
    for batch_idx, b in enumerate(test_loader):
        imgs = b[0]
        captions = b[1]
        
        img_feature, caption_feature  = imgs, [caption.to(device) for caption in captions]
        
        model(img_feature, caption_feature)
        h_image, h_caption = model(img_feature, caption_feature)
        l = h_image.shape[0]
        
        
        S_image = -1*torch.ones((l,1)).cuda()
        S_caption = -1*torch.ones((l,1)).cuda()
        accuracy = 0
        
        for i in range(l):
            index_image = 0
            for j in range(l):
                if S_image[i] <  h_image[i].dot(h_caption[j]):
                    S_image[i] =  h_image[i].dot(h_caption[j])
                    index_image = j
                    
            if i == index_image:
                accuracy += 1.0
            
        
        
        for i in range(l):
            index_caption = 0
            for j in range(l):    
                if S_caption[i] <  h_caption[i].dot(h_image[j]):
                    S_caption[i] =  h_caption[i].dot(h_image[j])
                    index_caption = j
                    
            if i == index_caption:
                accuracy += 1.0
           
        
        accuracy /= float(2*l)

        return accuracy

In [7]:
# optimizer = optim.SGD(model.parameters(), 0.01,momentum=0.9,weight_decay=1e-4)
optimizer = optim.Adam(model.parameters())

def my_collate(batch):
    img = [item["features"] for item in batch]
    caption = [item["captions"] for item in batch]
    #caption = torch.LongTensor(caption)
    return [img, caption]

train_loader = DataLoader( CAD120Dataset('features.bin','caption.bin'), batch_size=128, shuffle = True,
                          collate_fn = my_collate)
test_loader = DataLoader( CAD120Dataset('features_subject3.bin','caption_subject3.bin'), batch_size=128, shuffle = True,
                          collate_fn = my_collate)

In [8]:
for epoch in range(1000):
    loss,accuracy = train(model, 'cuda:0', train_loader, optimizer, epoch)
    writer.add_scalars('data/loss', {'loss': loss}, epoch)
    writer.add_scalars('data/train_accuracy', {'train_accuracy': accuracy}, epoch)
    accuracy = test(model, 'cuda:0', test_loader,epoch)
    writer.add_scalars('data/test_accuracy', {'test_accuracy': accuracy}, epoch)
    if epoch == 500:
        optimizer = optim.SGD(model.parameters(), 0.01,momentum=0.9,weight_decay=1e-4)
writer.export_scalars_to_json("./all_scalars.json")
writer.close()

Train Epoch: 0 [0/164 (0%)]	Loss: 8.085978
Train Epoch: 1 [0/164 (0%)]	Loss: 10.471212
Train Epoch: 2 [0/164 (0%)]	Loss: 10.587696
Train Epoch: 3 [0/164 (0%)]	Loss: 8.652025
Train Epoch: 4 [0/164 (0%)]	Loss: 6.220052
Train Epoch: 5 [0/164 (0%)]	Loss: 4.639176
Train Epoch: 6 [0/164 (0%)]	Loss: 4.771930
Train Epoch: 7 [0/164 (0%)]	Loss: 4.377268
Train Epoch: 8 [0/164 (0%)]	Loss: 3.662331
Train Epoch: 9 [0/164 (0%)]	Loss: 3.933893
Train Epoch: 10 [0/164 (0%)]	Loss: 3.483151
Train Epoch: 11 [0/164 (0%)]	Loss: 3.933923
Train Epoch: 12 [0/164 (0%)]	Loss: 3.350154
Train Epoch: 13 [0/164 (0%)]	Loss: 3.173579
Train Epoch: 14 [0/164 (0%)]	Loss: 3.202273
Train Epoch: 15 [0/164 (0%)]	Loss: 3.415352
Train Epoch: 16 [0/164 (0%)]	Loss: 3.084725
Train Epoch: 17 [0/164 (0%)]	Loss: 2.876230
Train Epoch: 18 [0/164 (0%)]	Loss: 2.686085
Train Epoch: 19 [0/164 (0%)]	Loss: 2.670209
Train Epoch: 20 [0/164 (0%)]	Loss: 2.609627
Train Epoch: 21 [0/164 (0%)]	Loss: 2.557216
Train Epoch: 22 [0/164 (0%)]	Loss: 2.510

Train Epoch: 185 [0/164 (0%)]	Loss: 2.008194
Train Epoch: 186 [0/164 (0%)]	Loss: 2.009662
Train Epoch: 187 [0/164 (0%)]	Loss: 2.009001
Train Epoch: 188 [0/164 (0%)]	Loss: 2.007603
Train Epoch: 189 [0/164 (0%)]	Loss: 2.009684
Train Epoch: 190 [0/164 (0%)]	Loss: 2.011227
Train Epoch: 191 [0/164 (0%)]	Loss: 2.013406
Train Epoch: 192 [0/164 (0%)]	Loss: 2.008060
Train Epoch: 193 [0/164 (0%)]	Loss: 2.009459
Train Epoch: 194 [0/164 (0%)]	Loss: 2.009907
Train Epoch: 195 [0/164 (0%)]	Loss: 2.009729
Train Epoch: 196 [0/164 (0%)]	Loss: 2.008651
Train Epoch: 197 [0/164 (0%)]	Loss: 2.006269
Train Epoch: 198 [0/164 (0%)]	Loss: 2.007973
Train Epoch: 199 [0/164 (0%)]	Loss: 2.008628
Train Epoch: 200 [0/164 (0%)]	Loss: 2.012275
Train Epoch: 201 [0/164 (0%)]	Loss: 2.007712
Train Epoch: 202 [0/164 (0%)]	Loss: 2.012097
Train Epoch: 203 [0/164 (0%)]	Loss: 2.006772
Train Epoch: 204 [0/164 (0%)]	Loss: 2.008028
Train Epoch: 205 [0/164 (0%)]	Loss: 2.008667
Train Epoch: 206 [0/164 (0%)]	Loss: 2.009953
Train Epoc

Train Epoch: 368 [0/164 (0%)]	Loss: 2.004986
Train Epoch: 369 [0/164 (0%)]	Loss: 2.005018
Train Epoch: 370 [0/164 (0%)]	Loss: 2.004896
Train Epoch: 371 [0/164 (0%)]	Loss: 2.004310
Train Epoch: 372 [0/164 (0%)]	Loss: 2.004810
Train Epoch: 373 [0/164 (0%)]	Loss: 2.004636
Train Epoch: 374 [0/164 (0%)]	Loss: 2.004288
Train Epoch: 375 [0/164 (0%)]	Loss: 2.003590
Train Epoch: 376 [0/164 (0%)]	Loss: 2.004028
Train Epoch: 377 [0/164 (0%)]	Loss: 2.004526
Train Epoch: 378 [0/164 (0%)]	Loss: 2.004315
Train Epoch: 379 [0/164 (0%)]	Loss: 2.003213
Train Epoch: 380 [0/164 (0%)]	Loss: 2.002918
Train Epoch: 381 [0/164 (0%)]	Loss: 2.004591
Train Epoch: 382 [0/164 (0%)]	Loss: 2.003964
Train Epoch: 383 [0/164 (0%)]	Loss: 2.003204
Train Epoch: 384 [0/164 (0%)]	Loss: 2.004864
Train Epoch: 385 [0/164 (0%)]	Loss: 2.004858
Train Epoch: 386 [0/164 (0%)]	Loss: 2.004658
Train Epoch: 387 [0/164 (0%)]	Loss: 2.004139
Train Epoch: 388 [0/164 (0%)]	Loss: 2.004949
Train Epoch: 389 [0/164 (0%)]	Loss: 2.003470
Train Epoc

Train Epoch: 551 [0/164 (0%)]	Loss: 2.004121
Train Epoch: 552 [0/164 (0%)]	Loss: 2.003918
Train Epoch: 553 [0/164 (0%)]	Loss: 2.003604
Train Epoch: 554 [0/164 (0%)]	Loss: 2.003514
Train Epoch: 555 [0/164 (0%)]	Loss: 2.003840
Train Epoch: 556 [0/164 (0%)]	Loss: 2.003237
Train Epoch: 557 [0/164 (0%)]	Loss: 2.003343
Train Epoch: 558 [0/164 (0%)]	Loss: 2.003811
Train Epoch: 559 [0/164 (0%)]	Loss: 2.003682
Train Epoch: 560 [0/164 (0%)]	Loss: 2.002959
Train Epoch: 561 [0/164 (0%)]	Loss: 2.002419
Train Epoch: 562 [0/164 (0%)]	Loss: 2.004573
Train Epoch: 563 [0/164 (0%)]	Loss: 2.002921
Train Epoch: 564 [0/164 (0%)]	Loss: 2.003919
Train Epoch: 565 [0/164 (0%)]	Loss: 2.004530
Train Epoch: 566 [0/164 (0%)]	Loss: 2.003013
Train Epoch: 567 [0/164 (0%)]	Loss: 2.004884
Train Epoch: 568 [0/164 (0%)]	Loss: 2.003314
Train Epoch: 569 [0/164 (0%)]	Loss: 2.002717
Train Epoch: 570 [0/164 (0%)]	Loss: 2.002856
Train Epoch: 571 [0/164 (0%)]	Loss: 2.003328
Train Epoch: 572 [0/164 (0%)]	Loss: 2.003671
Train Epoc

Train Epoch: 734 [0/164 (0%)]	Loss: 2.002004
Train Epoch: 735 [0/164 (0%)]	Loss: 2.002334
Train Epoch: 736 [0/164 (0%)]	Loss: 2.002427
Train Epoch: 737 [0/164 (0%)]	Loss: 2.002482
Train Epoch: 738 [0/164 (0%)]	Loss: 2.003137
Train Epoch: 739 [0/164 (0%)]	Loss: 2.003694
Train Epoch: 740 [0/164 (0%)]	Loss: 2.002964
Train Epoch: 741 [0/164 (0%)]	Loss: 2.002134
Train Epoch: 742 [0/164 (0%)]	Loss: 2.003814
Train Epoch: 743 [0/164 (0%)]	Loss: 2.003845
Train Epoch: 744 [0/164 (0%)]	Loss: 2.002699
Train Epoch: 745 [0/164 (0%)]	Loss: 2.004134
Train Epoch: 746 [0/164 (0%)]	Loss: 2.003048
Train Epoch: 747 [0/164 (0%)]	Loss: 2.003639
Train Epoch: 748 [0/164 (0%)]	Loss: 2.005617
Train Epoch: 749 [0/164 (0%)]	Loss: 2.004646
Train Epoch: 750 [0/164 (0%)]	Loss: 2.002283
Train Epoch: 751 [0/164 (0%)]	Loss: 2.004163
Train Epoch: 752 [0/164 (0%)]	Loss: 2.003064
Train Epoch: 753 [0/164 (0%)]	Loss: 2.004308
Train Epoch: 754 [0/164 (0%)]	Loss: 2.003925
Train Epoch: 755 [0/164 (0%)]	Loss: 2.003099
Train Epoc

Train Epoch: 917 [0/164 (0%)]	Loss: 2.002697
Train Epoch: 918 [0/164 (0%)]	Loss: 2.002179
Train Epoch: 919 [0/164 (0%)]	Loss: 2.001590
Train Epoch: 920 [0/164 (0%)]	Loss: 2.002476
Train Epoch: 921 [0/164 (0%)]	Loss: 2.002438
Train Epoch: 922 [0/164 (0%)]	Loss: 2.001556
Train Epoch: 923 [0/164 (0%)]	Loss: 2.002950
Train Epoch: 924 [0/164 (0%)]	Loss: 2.002687
Train Epoch: 925 [0/164 (0%)]	Loss: 2.002973
Train Epoch: 926 [0/164 (0%)]	Loss: 2.001989
Train Epoch: 927 [0/164 (0%)]	Loss: 2.002573
Train Epoch: 928 [0/164 (0%)]	Loss: 2.002263
Train Epoch: 929 [0/164 (0%)]	Loss: 2.002516
Train Epoch: 930 [0/164 (0%)]	Loss: 2.002731
Train Epoch: 931 [0/164 (0%)]	Loss: 2.003859
Train Epoch: 932 [0/164 (0%)]	Loss: 2.002791
Train Epoch: 933 [0/164 (0%)]	Loss: 2.002367
Train Epoch: 934 [0/164 (0%)]	Loss: 2.003780
Train Epoch: 935 [0/164 (0%)]	Loss: 2.003611
Train Epoch: 936 [0/164 (0%)]	Loss: 2.002684
Train Epoch: 937 [0/164 (0%)]	Loss: 2.003366
Train Epoch: 938 [0/164 (0%)]	Loss: 2.003912
Train Epoc